In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [2]:
movies = pd.read_csv('D:/ml-latest/ml-latest/movies.csv')

In [4]:
#genome_scores = pd.read_csv('D:/ml-latest/ml-latest/genome-scores.csv')
#genome_tags = pd.read_csv('D:/ml-latest/ml-latest/genome-tags.csv')
ratings = pd.read_csv('D:/ml-latest/ml-latest/ratings.csv')
#tags = pd.read_csv('D:/ml-latest/ml-latest/tags.csv')

In [5]:
merged_genome = genome_scores.merge(genome_tags, on='tagId', how='left')

In [21]:
merged_movies = movies.merge(merged_genome, on='movieId', how='left')

In [22]:
merged_movies.head(10)

,movieId,title,genres,tagId,relevance,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,0.03200,007
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.0,0.02225,007 (series)
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,0.07000,18th century
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,0.05900,1920s
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,0.12300,1930s
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,0.13100,1950s
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,0.06175,1960s
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,0.19550,1970s
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,0.26625,1980s
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,0.03300,19th century


In [7]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [13]:
ratings[["userId", "movieId", "rating"]].head()

,userId,movieId,rating
0,1,1,4.0
1,1,110,4.0
2,1,158,4.0
3,1,260,4.5
4,1,356,5.0


In [9]:
tags['timestamp'] = pd.to_datetime(tags['timestamp'], unit='s')

In [10]:
tags.head()

,userId,movieId,tag,timestamp
0,10,260,good vs evil,2015-05-03 15:22:38
1,10,260,Harrison Ford,2015-05-03 15:21:45
2,10,260,sci-fi,2015-05-03 15:22:18
3,14,1221,Al Pacino,2011-07-25 13:32:36
4,14,1221,mafia,2011-07-25 13:32:26


Задача 1

In [11]:
# Агрегируем данные по фильмам: считаем среднюю оценку и количество оценок для каждого фильма
agg_ratings = ratings.groupby('movieId').agg({
    'rating': ['mean', 'count']
})
agg_ratings.columns = ['vote_average', 'vote_count']  # Переименовываем столбцы для удобства
agg_ratings.reset_index(inplace=True)

In [ ]:
movies_with_ratings = movies.merge(agg_ratings, on='movieId')

Продвижение фильмов

In [59]:
#Создаем дополнительный столюец flag, где 1 - продвижение, 0 - сохранение и -1 снижение 
movies_with_ratings['flag'] = 0

Присваем значения колнки флаг выбранным фильмам

In [63]:
movies_with_ratings.loc[movies_with_ratings['title'] == 'Ouija Japan (2021)', 'vote_count'] = 1


In [64]:
movies_with_ratings.loc[movies_with_ratings['title'] == 'Ouija Japan (2021)', 'flag'] = 1


In [74]:
movies_with_ratings.loc[movies_with_ratings['title'] == 'Godfather: Part II, The (1974)', 'flag'] = -1

In [80]:
movies_with_ratings.head(10)

,movieId,title,genres,vote_average,vote_count,score,flag
314,318,"Shawshank Redemption, The (1994)",Crime|Drama,4.416792,122296,4.413779,0
39322,159817,Planet Earth (2006),Documentary,4.448093,3015,4.332929,0
840,858,"Godfather, The (1972)",Crime|Drama,4.326603,75004,4.322012,0
44066,170705,Band of Brothers (2001),Action|Drama|War,4.423986,2835,4.304177,0
58750,202439,Parasite (2019),Comedy|Drama,4.329946,12399,4.302586,0
44214,171011,Planet Earth II (2016),Documentary,4.451739,2041,4.287475,0
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.267865,72893,4.263353,0
1190,1221,"Godfather: Part II, The (1974)",Crime|Drama,4.269510,47271,4.262556,-1
1173,1203,12 Angry Men (1957),Drama,4.267158,22730,4.252809,0
522,527,Schindler's List (1993),Drama|War,4.242337,84232,4.238509,0


In [81]:
m = movies_with_ratings['vote_count'].quantile(0.9)

In [82]:
C = movies_with_ratings['vote_average'].mean()

Добавляем условия для фильтрации фильмов с учетом колонки flag

In [83]:
qualified_movies = movies_with_ratings[(movies_with_ratings['vote_count'] >= m) | (movies_with_ratings['flag'] == 1)].copy()

Функция для подбора лучших фильмов в случае холодного старта

In [54]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C) 

Доработанная функция для продвижения

In [88]:
def weights_films(x, m=m, C=C):
    flag = x['flag']
    v = x['vote_count']
    R = x['vote_average']
    
    if flag == 1:
        return (v/(v+m) * R) + (m/(m+v) * C) * 1000
    elif flag == -1:
        return ((v/(v+m) * R) + (m/(m+v) * C)) * -1
    else:
        return (v/(v+m) * R) + (m/(m+v) * C) 

In [89]:
# Применяем нашу функцию к каждой строке
qualified_movies['score'] = qualified_movies.apply(weights_films, axis=1)

# Сортируем фильмы по рассчитанному взвешенному рейтингу
qualified_movies = qualified_movies.sort_values('score', ascending=False)

# Выводим топ-10 фильмов с их названием, средней оценкой, количеством оценок и итоговым рейтингом
print(qualified_movies[['title', 'vote_average', 'vote_count', 'score', 'genres']].head(20))

                                                   title  vote_average  \
83235                                 Ouija Japan (2021)      0.500000   
314                     Shawshank Redemption, The (1994)      4.416792   
39322                                Planet Earth (2006)      4.448093   
840                                Godfather, The (1972)      4.326603   
44066                            Band of Brothers (2001)      4.423986   
58750                                    Parasite (2019)      4.329946   
44214                             Planet Earth II (2016)      4.451739   
49                            Usual Suspects, The (1995)      4.267865   
1173                                 12 Angry Men (1957)      4.267158   
522                              Schindler's List (1993)      4.242337   
2867                                   Fight Club (1999)      4.236019   
1930         Seven Samurai (Shichinin no samurai) (1954)      4.250818   
5509   Spirited Away (Sen to Chihiro n

Первоначальный топ

Shawshank Redemption, The (1994)      4.416792      122296   
840                     Godfather, The (1972)      4.326603       75004   
49                 Usual Suspects, The (1995)      4.267865       72893   
522                   Schindler's List (1993)      4.242337       84232   
2867                        Fight Club (1999)      4.236019       86207   
1190           Godfather: Part II, The (1974)      4.269510       47271   
292                       Pulp Fiction (1994)      4.191778      108756   
1164   One Flew Over the Cuckoo's Nest (1975)      4.212801       49316   
12221                 Dark Knight, The (2008)      4.187539       65349   
2480                       Matrix, The (1999)      4.160631      107056   

Задача 2

Выбираем фильмы с релевантными тегами

In [18]:
merged_movies_rel = merged_movies[merged_movies['relevance'] > 0.6]

In [19]:
merged_movies_rel.shape

(425860, 6)

In [5]:
# Создаем экземпляр MinMaxScaler
scaler = MinMaxScaler()

In [6]:
def get_recommendations(user_ratings: dict, ratings_df: pd.DataFrame, movies_df: pd.DataFrame, top_n: int):

    # 1. Создаем DataFrame с оценками текущего пользователя
    user_ratings_series = pd.Series(user_ratings)
    user_df = pd.DataFrame(user_ratings_series, columns=['rating'])
    user_df['userId'] = 'current_user'
    user_df = user_df.reset_index().rename(columns={'index': 'movieId'})
    user_df['movieId'] = pd.to_numeric(user_df['movieId'])

    # Объединяем оценки текущего пользователя с общим датасетом оценок
    all_ratings_df = pd.concat([ratings_df, user_df], ignore_index=True)

    #Нормализация оценки
    all_ratings_df['normalized_rating'] = scaler.fit_transform(all_ratings_df[['rating']])
    all_ratings_df['rating'] = all_ratings_df['normalized_rating']
    all_ratings_df.drop(columns=['normalized_rating'], inplace=True)

    
    # 2. Создаем матрицу взаимодействия пользователь-фильм
    user_movie_matrix = all_ratings_df.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

    # Находим вектор текущего пользователя
    current_user_vector = user_movie_matrix.loc['current_user']

    # Рассчитываем сходство между текущим пользователем и другими пользователями
    user_similarity = cosine_similarity(current_user_vector.values.reshape(1, -1), user_movie_matrix.drop('current_user').values)[0]
    user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.drop('current_user').index, columns=['similarity'])

    # 3. Получаем фильмы, которые смотрели похожие пользователи, и взвешиваем их оценки
    similar_users_ratings = all_ratings_df[all_ratings_df['userId'].isin(user_similarity_df.nlargest(5, 'similarity').index)]
    weighted_ratings = similar_users_ratings.groupby('movieId')['rating'].mean() 

    # 4. Исключаем фильмы, которые уже смотрел текущий пользователь
    rated_movie_ids = user_ratings.keys()
    recommendations = weighted_ratings[~weighted_ratings.index.isin(rated_movie_ids)].sort_values(ascending=False)

    # 5. Объединяем с информацией о фильмах для получения названия и других деталей
    recommendation_df = pd.DataFrame(recommendations).reset_index().rename(columns={'rating': 'predicted_rating'})
    recommended_movies_info = pd.merge(recommendation_df, movies_df, on='movieId', how='inner')

    return recommended_movies_info.head(top_n)

In [15]:
user_watched = {16: 5, 13: 4, 2: 3, 288: 5}
top_recommendations = get_recommendations(user_watched, ratings[['userId', 'movieId', 'rating']][:1000], 
                                          movies[['movieId', 'title', 'genres']][:1000], top_n=3)

predicted_rating нормализован поэтому 1 значит высшую оценку

In [16]:
from tabulate import tabulate

print("Рекомендации для пользователя:")
print(tabulate(top_recommendations, headers='keys', tablefmt='grid'))

Рекомендации для пользователя:
+----+-----------+--------------------+-----------------------+-------------+
|    |   movieId |   predicted_rating | title                 | genres      |
+====+===========+====================+=======================+=============+
|  0 |       457 |                  1 | Fugitive, The (1993)  | Thriller    |
+----+-----------+--------------------+-----------------------+-------------+
|  1 |       141 |                  1 | Birdcage, The (1996)  | Comedy      |
+----+-----------+--------------------+-----------------------+-------------+
|  2 |       858 |                  1 | Godfather, The (1972) | Crime|Drama |
+----+-----------+--------------------+-----------------------+-------------+
